In [35]:
import json 
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from datetime import datetime


SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd()


In [36]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [37]:
#Data
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

Archive:  /content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip
replace MaskDataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/MaskDataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [39]:
#datagenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator


apply_data_augmentation = False

# Create training ImageDataGenerator object

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        vertical_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

#set directories
dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir= os.path.join(dataset_dir,"training")

#Batch size
BS = 64

#Image shape
IMG_H = 407 #612
IMG_W = 407

num_classes=3


with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic=json.load(f)
  dataframe= pd.DataFrame(dic.items())
  dataframe.rename(columns={0:'filename', 1:'class'}, inplace=True )
  dataframe=dataframe.sample(frac=1, random_state=SEED)
  dataframe["class"]=dataframe["class"].astype('string')
  train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 seed=SEED,
                                                 subset='training',
                                                 image_size=(IMG_H,IMG_W))
  
  validation_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle=False,
                                                 seed=SEED,
                                                 subset='validation',
                                                 image_size=(IMG_H,IMG_W))
  




Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [40]:
#create Dataset Object

#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))


train_dataset = train_dataset.repeat()

#Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()


In [41]:
# Architecture: Features extraction -> Classifier

start_f = 25 # number of feature 
depth = 8 #how many encoding block 

model = tf.keras.Sequential() #sequential model

# Features extraction 
for i in range(depth):
    if i == 0:  #at the first layer I have to define the input shape as a list. so my input is the image dimension 
        input_shape = [IMG_H, IMG_W, 3]
    else:
        input_shape=[None] #otherwise we left as unknown 

    #Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU()) #activation layer
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    #that is 2,2 -> so halved the dimension 

    start_f *= 2

# Classifier
#I vectorize the volume and apply classifier 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2, seed=SEED))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [42]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights 

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 407, 407, 25)      700       
_________________________________________________________________
re_lu_84 (ReLU)              (None, 407, 407, 25)      0         
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 203, 203, 25)      0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 203, 203, 50)      11300     
_________________________________________________________________
re_lu_85 (ReLU)              (None, 203, 203, 50)      0         
_________________________________________________________________
max_pooling2d_85 (MaxPooling (None, 101, 101, 50)      0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 101, 101, 100)   

In [43]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [44]:
#Callbacks and checkpoints

cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'First'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

#Model checkpoint
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_best_only=True, 
                                                   save_freq = 142,
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6,restore_best_weights = True )
    callbacks.append(es_callback)

In [45]:
model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(validation_gen),
          callbacks=callbacks)


Epoch 1/30
71/71 [==============================] - 79s 1s/step - loss: 1.0998 - accuracy: 0.3344 - val_loss: 1.0985 - val_accuracy: 0.3307
Epoch 2/30
71/71 [==============================] - 77s 1s/step - loss: 1.0986 - accuracy: 0.3428 - val_loss: 1.0981 - val_accuracy: 0.3333
Epoch 3/30
71/71 [==============================] - 77s 1s/step - loss: 1.0981 - accuracy: 0.3522 - val_loss: 1.0979 - val_accuracy: 0.3378
Epoch 4/30
71/71 [==============================] - 76s 1s/step - loss: 1.0582 - accuracy: 0.4325 - val_loss: 0.9639 - val_accuracy: 0.5107
Epoch 5/30
71/71 [==============================] - 76s 1s/step - loss: 0.8171 - accuracy: 0.6049 - val_loss: 0.7202 - val_accuracy: 0.6667
Epoch 6/30
71/71 [==============================] - 75s 1s/step - loss: 0.6777 - accuracy: 0.6805 - val_loss: 0.6535 - val_accuracy: 0.6988
Epoch 7/30
71/71 [==============================] - 76s 1s/step - loss: 0.5876 - accuracy: 0.7231 - val_loss: 0.5788 - val_accuracy: 0.7201
Epoch 8/30
71/71 [==

In [46]:
#create prediction and csv
from os import listdir

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1





Found 450 validated image filenames belonging to 1 classes.
8/8 [==============================] - 3s 427ms/step


In [47]:
create_csv(results, '/content/drive/My Drive')